<a href="https://colab.research.google.com/github/laresistance-bolivia/BOLIVIA_ELECCIONES_2019/blob/master/ANALISIS_TREP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis del TREP para las elecciones presidenciales 2019 de Bolivia

- De preferencia corre el notebook en Google Colab

- La fuente de datos se descarga de manera automática de la siguiente fuente:
https://drive.google.com/drive/folders/1z-Jwked3bX3ZiHEuy8ckvnSzRwAz62RF

- La mayor parte de los gráficos son interactivos, puedes habilitar y deshabilitar opciones desde el menú lateral y hacer zoom sobre los mismos.



In [0]:
# Download DB
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1StCAGlX-ddYqFcdmtQWDUCuRRqKrfGgL' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1StCAGlX-ddYqFcdmtQWDUCuRRqKrfGgL" -O historico_actas.zip && rm -rf /tmp/cookies.txt

In [1]:
!unzip historico_actas.zip

Archive:  historico_actas.zip
  inflating: DatosActasPorHora.db    


In [2]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.express as px
init_notebook_mode(connected=True)

In [0]:
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

In [0]:
# DataBase connection
conn = sqlite3.connect("DatosActasPorHora.db")

In [10]:
# Table Description
query = "PRAGMA table_info(ActasPresidenteTrep);"
pd.read_sql_query(query, conn)

,cid,name,type,notnull,dflt_value,pk
0,0,Id,INTEGER,1,None,1
1,1,FechaHoraArchivo,TEXT,1,None,0
2,2,Pais,TEXT,0,None,0
3,3,NumeroDepartamento,INTEGER,1,None,0
4,4,Provincia,TEXT,0,None,0
5,5,NumeroMunicipio,INTEGER,1,None,0
6,6,Municipio,TEXT,0,None,0
7,7,Circunscripcion,TEXT,0,None,0
8,8,Localidad,TEXT,0,None,0
9,9,Recinto,TEXT,0,None,0


In [12]:
# Registros por archivo
query = """
SELECT FechaHoraArchivo, count(*) as total_registros
FROM ActasPresidenteTrep 
group by FechaHoraArchivo
"""
df_actas_qty = pd.read_sql_query(query, conn)
df_actas_qty['FechaHoraArchivo'] = pd.to_datetime(df_actas_qty['FechaHoraArchivo'])
df_actas_qty

,FechaHoraArchivo,total_registros
0,2019-10-20 19:40:57,28975
1,2019-10-21 19:14:53,32849
2,2019-10-21 20:17:53,32920
3,2019-10-21 21:20:53,33013
4,2019-10-21 22:23:53,33044
...,...,...
80,2019-10-25 06:01:40,33044
81,2019-10-25 07:04:40,33044
82,2019-10-25 08:07:40,33044
83,2019-10-25 09:10:40,33044


In [0]:
# df_actas_qty.plot(x="FechaHoraArchivo", y="total_registros")

## Cantidad de Registros en Actas

In [16]:
configure_plotly_browser_state()
data = go.Scatter(
           x=df_actas_qty['FechaHoraArchivo'],
           y=df_actas_qty['total_registros'])

layout = go.Layout(title='Cantidad de Registros en actas', 
                   xaxis=dict(title='Fecha'),
                   yaxis=dict(title='Cantidad'))

fig = go.Figure(data, layout=layout)
iplot(fig)

##### Cálculos

In [0]:
cols_partidos = ['CC', 'FPV', 'MTS', 'UCS', 'MAS', 'M21F', 'PDC', 'MNR', 'PanBol']

In [0]:
# Obtener todas las actas
query = "SELECT * FROM ActasPresidenteTrep"
df_actas_all = pd.read_sql_query(query, conn)
df_actas_all['FechaHoraArchivo'] = pd.to_datetime(df_actas_all['FechaHoraArchivo'])
df_actas_all.sort_values( by=['FechaHoraArchivo'], inplace=True)

In [0]:
# Solo valores numéricos
# df_actas_all_numericos = df_actas_all[['CodigoMesa', 'FechaHoraArchivo' ] + cols_partidos + ['Validos', 'Blancos', 'Nulos', 'Inscritos']]
# df_actas_all_numericos.head()

In [20]:
# Cálculo de votos válidos
df_actas_all['_ValidosCalculado'] = df_actas_all[cols_partidos].sum(axis=1)
df_actas_all['_Validos_menos_ValidosCalculado'] = df_actas_all['Validos'] - df_actas_all['_ValidosCalculado']
df_actas_all['_Validos_menos_ValidosCalculado_Abs'] = df_actas_all['_Validos_menos_ValidosCalculado'].abs()

# Ausentismo
df_actas_all['_Ausentismo_OEP'] = df_actas_all['Inscritos'] - (df_actas_all['Validos'] + df_actas_all['Blancos'] + df_actas_all['Nulos'])
df_actas_all['_Ausentismo_Calculado'] = df_actas_all['Inscritos'] - (df_actas_all['_ValidosCalculado'] + df_actas_all['Blancos'] + df_actas_all['Nulos'])

# Total de Votantes por mesa
df_actas_all['_Votantes'] = df_actas_all['_ValidosCalculado'] + df_actas_all['Blancos'] + df_actas_all['Nulos']

# Asistencia Total en mesas
df_actas_all['_TotalAsistencia'] = 'NO'
df_actas_all.loc[df_actas_all['_Votantes'] == df_actas_all['Inscritos'] , '_TotalAsistencia'] = 'SI' 


# Mesa Ganadora
df_actas_all['_PartidoGanador'] = df_actas_all[cols_partidos].idxmax(axis=1)


df_actas_all.head()

,Id,FechaHoraArchivo,Pais,NumeroDepartamento,Provincia,NumeroMunicipio,Municipio,Circunscripcion,Localidad,Recinto,NumeroMesa,CodigoMesa,Eleccion,Inscritos,CC,FPV,MTS,UCS,MAS,M21F,PDC,MNR,PanBol,Validos,Blancos,Nulos,Computada,Departamento,_ValidosCalculado,_Validos_menos_ValidosCalculado,_Validos_menos_ValidosCalculado_Abs,_Ausentismo_OEP,_Ausentismo_Calculado,_Votantes,_TotalAsistencia,_PartidoGanador
0,1,2019-10-20 19:40:57,Alemania,61,Berlín,1,Berlín,,Berlín,Embajada,1001,110046138042512901,Presidente y Vicepresidente,237,129,0,3,0,37,0,1,1,2,173,1,0,None,Berlín,173,0,0,63,63,174,NO,CC
19324,19325,2019-10-20 19:40:57,Bolivia,6,Arce,2,Bermejo,,Bermejo,U.E. Octavio Campero Echazu (Bermejo),60962,100320619160001418,Presidente y Vicepresidente,212,66,0,0,1,67,3,19,5,1,162,3,9,None,Tarija,162,0,0,38,38,174,NO,MAS
19323,19324,2019-10-20 19:40:57,Bolivia,6,Arce,2,Bermejo,,Bermejo,U.E. Octavio Campero Echazu (Bermejo),60956,100320619160001412,Presidente y Vicepresidente,231,70,0,0,3,77,5,23,3,1,182,4,15,None,Tarija,182,0,0,30,30,201,NO,MAS
19322,19323,2019-10-20 19:40:57,Bolivia,6,Arce,2,Bermejo,,Bermejo,U.E. Octavio Campero Echazu (Bermejo),60955,100320619160001411,Presidente y Vicepresidente,228,67,1,1,0,92,4,15,2,4,186,3,14,None,Tarija,186,0,0,25,25,203,NO,MAS
19321,19322,2019-10-20 19:40:57,Bolivia,6,Arce,2,Bermejo,,Bermejo,U.E. Octavio Campero Echazu (Bermejo),60954,100320619160001410,Presidente y Vicepresidente,229,74,0,0,2,78,2,21,4,0,181,2,17,None,Tarija,181,0,0,29,29,200,NO,MAS


### Ultimo archivo disponible

In [21]:
# Acta final (En base a la fecha máxima)
acta_final_fecha = str(df_actas_all['FechaHoraArchivo'].max())
acta_final_fecha

'2019-10-25 10:13:40'

In [22]:
# Cálculos sobre la última acta
print("Cálculos sobre la última acta")
# Errores de calculo de sumatoria votos validos  en la ultima acta 
df_validos_mal_computados = df_actas_all[(df_actas_all['FechaHoraArchivo'] == acta_final_fecha) & (df_actas_all['_Validos_menos_ValidosCalculado_Abs'] != 0)].groupby(by=['Departamento', 'Municipio'] ).sum()[['_Validos_menos_ValidosCalculado_Abs', '_Ausentismo_Calculado']].reset_index()

## Ausentismo ultima acta
df_ausentismo_calculado = df_actas_all[(df_actas_all['FechaHoraArchivo'] == acta_final_fecha) & (df_actas_all['_Ausentismo_Calculado'] != 0)].groupby(by=['Departamento', 'Municipio', '_PartidoGanador'] ).sum()[['_Ausentismo_Calculado']].reset_index()

## Resumen 
df_actas_inscritos = df_actas_all[(df_actas_all['FechaHoraArchivo'] == acta_final_fecha)].groupby(by=['Pais', 'Departamento', 'Provincia']).sum()[['Inscritos', '_ValidosCalculado', '_Ausentismo_Calculado', 'MAS', 'CC']].reset_index()
df_actas_inscritos.sample(5)

Cálculos sobre la última acta


,Pais,Departamento,Provincia,Inscritos,_ValidosCalculado,_Ausentismo_Calculado,MAS,CC
117,Bolivia,Pando,Manuripi,8283,6595,1202,3537,2141
13,Argentina,Buenos Aires,Pinamar,1226,954,220,805,49
166,Chile,Arica,Arica,3150,2337,710,1614,382
108,Bolivia,Oruro,Sajama,4737,3661,851,2607,330
16,Argentina,Chubut,Escalante,1796,1243,480,1151,43


## Error de suma en votos válidos por municipio

In [23]:
configure_plotly_browser_state()
fig = px.bar(df_validos_mal_computados, x="Municipio", y="_Validos_menos_ValidosCalculado_Abs", title="Votos válidos mal sumados")
fig.show()


## Mesas con asistencia total

In [24]:
## Mesas con total asistencia
# Corrigieron esto en la última version de las actas
df_total_asistencia = df_actas_all[(df_actas_all['FechaHoraArchivo'] == acta_final_fecha) & (df_actas_all['_TotalAsistencia'] == 'SI')].groupby(by=['Departamento', 'Provincia', 'Municipio', '_PartidoGanador'] ).sum()[['Inscritos', 'MAS', 'CC']].reset_index()

# df_total_asistencia = df_actas_all[df_actas_all['_TotalAsistencia'] == 'SI'].groupby(by=['FechaHoraArchivo', 'Departamento', 'Provincia', '_PartidoGanador'] ).sum()[['Inscritos', 'MAS', 'CC']].reset_index()
print(len(df_total_asistencia))
print("Muestra de 10 mesas")
df_total_asistencia.sample(10)

129
Muestra de 10 mesas


,Departamento,Provincia,Municipio,_PartidoGanador,Inscritos,MAS,CC
71,Pando,Nicolas Suarez,Porvenir,MAS,47,30,5
8,Chuquisaca,Hernando Siles,Monteagudo,CC,18,1,14
66,Oruro,Sabaya,Sabaya,CC,625,85,394
68,Pando,Manuripi,Filadelfia,MAS,37,17,14
38,La Paz,General José Manuel Pando,Santiago de Machaca,MAS,57,46,0
40,La Paz,Gualberto Villarroel,San Pedro de Curahuara,MAS,223,213,1
17,Cochabamba,Carrasco,Pojo,MAS,196,169,1
64,Oruro,Ladislao Cabrera,Salinas de Garcí Mendoza,MAS,54,45,5
7,Chuquisaca,Azurduy,Azurduy,MAS,23,21,0
49,La Paz,Murillo,El Alto,CC,225,80,103


### Mesas con asistencia total por municipios y partido ganador

In [25]:
configure_plotly_browser_state()
fig = px.bar(df_total_asistencia, x="Municipio", y="Inscritos", title="_Asistencia Total en Mesas", color="_PartidoGanador")
fig.show()


## Inscritos por Departamento

In [26]:
configure_plotly_browser_state()
fig = px.bar(df_actas_inscritos, x="Departamento", y="Inscritos",  title="Inscritos", color= "Pais")
fig.show()

## Ausentismo por departamento

In [27]:
configure_plotly_browser_state()
fig = px.bar(df_actas_inscritos, x="Departamento", y="_Ausentismo_Calculado",  title="Ausentismo", color= "Pais")
fig.show()

## Ausentismo por municipio y partido ganador

In [28]:
configure_plotly_browser_state()
fig = px.bar(df_ausentismo_calculado, x="Municipio", y="_Ausentismo_Calculado", title="Ausentismo", color='_PartidoGanador')
fig.show()

## Votos válidos por departamento

In [29]:
configure_plotly_browser_state()
fig = px.bar(df_actas_inscritos, x="Departamento", y="_ValidosCalculado",  title="Votos Válidos", color= "Pais")
fig.show()

## Votos válidos a favor del MAS

In [30]:
configure_plotly_browser_state()
fig = px.bar(df_actas_inscritos, x="Departamento", y="MAS",  title="Votos Válidos a favor del MAS", color= "Pais")
fig.show()

## Votos válidos a favor de CC

In [31]:
configure_plotly_browser_state()
fig = px.bar(df_actas_inscritos, x="Departamento", y="CC",  title="Votos Válidos a favor de CC", color= "Pais")
fig.show()

## Actas históricas analizadas

In [32]:
print("Total Actas del repositorio histórico")
len(df_actas_all)

Total Actas del repositorio histórico


2804321

In [33]:
df_actas_modificadas = df_actas_all [~df_actas_all.loc[:, ~df_actas_all.columns.isin(['FechaHoraArchivo', 'Id'])].duplicated(keep='first')]
print("Actas únicas con modificaciones")
len(df_actas_modificadas)
# df_actas_all.duplicated(keep='first', subset=[~df_actas_all.columns.isin(['FechaHoraArchivo', 'Id'])])

Actas únicas con modificaciones


33107

In [34]:
# Actas modificadas multiples veces
df_actas_modificadas_multiples_veces = df_actas_modificadas.groupby(by=['Pais', 'Departamento', 'Municipio', 'CodigoMesa']).count()['Inscritos'].reset_index()
df_actas_modificadas_multiples_veces = df_actas_modificadas_multiples_veces[df_actas_modificadas_multiples_veces['Inscritos'] > 1]
print("Actas modificadas más de una vez")
len(df_actas_modificadas_multiples_veces)

Actas modificadas más de una vez


1

In [35]:
print("Actas editadas")
df_actas_modificadas_multiples_veces.sort_values('Inscritos',  ascending=False).head(20)

Actas editadas


,Pais,Departamento,Municipio,CodigoMesa,Inscritos
32811,España,Islas Baleares,Palma de Mallorca,110673137742611705,2


In [36]:
df_mesas_corregidas_detalle = pd.DataFrame()
for mesa in df_actas_modificadas_multiples_veces['CodigoMesa']:
  # print(mesa)
  df_mesa = df_actas_all[df_actas_all['CodigoMesa'] == mesa].copy()
  df_mesa['Duplicated'] = df_mesa.duplicated(subset=df_mesa.columns[~df_mesa.columns.isin(['Id',	'FechaHoraArchivo'])])
  df_mesas_corregidas_detalle = df_mesas_corregidas_detalle.append(df_mesa[df_mesa['Duplicated'] == False], ignore_index=True)

df_mesas_corregidas_detalle.head(50)

,Id,FechaHoraArchivo,Pais,NumeroDepartamento,Provincia,NumeroMunicipio,Municipio,Circunscripcion,Localidad,Recinto,NumeroMesa,CodigoMesa,Eleccion,Inscritos,CC,FPV,MTS,UCS,MAS,M21F,PDC,MNR,PanBol,Validos,Blancos,Nulos,Computada,Departamento,_ValidosCalculado,_Validos_menos_ValidosCalculado,_Validos_menos_ValidosCalculado_Abs,_Ausentismo_OEP,_Ausentismo_Calculado,_Votantes,_TotalAsistencia,_PartidoGanador,Duplicated
0,1217,2019-10-20 19:40:57,España,31,Palma de Mallorca,1,Palma de Mallorca,,Palma de Mallorca,Centro Educativo Infantil y Primaria CEIP Aina...,2433,110673137742611705,Presidente y Vicepresidente,239,0,0,0,0,0,0,0,0,0,0,0,0,None,Islas Baleares,0,0,0,239,239,0,NO,CC,False
1,523849,2019-10-22 09:56:53,España,31,Palma de Mallorca,1,Palma de Mallorca,,Palma de Mallorca,Centro Educativo Infantil y Primaria CEIP Aina...,2433,110673137742611705,Presidente y Vicepresidente,239,74,1,1,0,39,2,14,1,1,133,0,5,None,Islas Baleares,133,0,0,101,101,138,NO,CC,False


In [0]:
 df_mesas_corregidas_detalle.to_csv('MesasCorregidas.csv')

In [0]:
#@title Mesa a buscar { run: "auto", form-width: "800px" }
mesa = "100320722341108805" #@param {type:"string"}
VER = "EDICIONES UNICAS" #@param ["TODOS", "EDICIONES UNICAS", "ULTIMA ACTA"]

df_mesa = df_actas_all[df_actas_all['CodigoMesa'] == mesa].copy()
if VER == "EDICIONES UNICAS":
  df_mesa['Duplicated'] = df_mesa.duplicated(subset=df_mesa.columns[~df_mesa.columns.isin(['Id',	'FechaHoraArchivo'])])
  display(df_mesa[df_mesa['Duplicated'] == False])

elif VER == "ULTIMA ACTA":
  display(df_actas_all[(df_actas_all['FechaHoraArchivo'] == acta_final_fecha) & (df_actas_all['CodigoMesa'] == mesa)])
else:
  display(df_mesa)

### Orden de Procesamiento de actas

In [38]:
df_llegada_actas = df_actas_all.drop_duplicates(subset=['CodigoMesa'], keep='first').copy()
df_llegada_actas.sort_values(by=["FechaHoraArchivo", "_PartidoGanador"], inplace=True)
print(len(df_llegada_actas))
df_llegada_actas.head()

33044


,Id,FechaHoraArchivo,Pais,NumeroDepartamento,Provincia,NumeroMunicipio,Municipio,Circunscripcion,Localidad,Recinto,NumeroMesa,CodigoMesa,Eleccion,Inscritos,CC,FPV,MTS,UCS,MAS,M21F,PDC,MNR,PanBol,Validos,Blancos,Nulos,Computada,Departamento,_ValidosCalculado,_Validos_menos_ValidosCalculado,_Validos_menos_ValidosCalculado_Abs,_Ausentismo_OEP,_Ausentismo_Calculado,_Votantes,_TotalAsistencia,_PartidoGanador
0,1,2019-10-20 19:40:57,Alemania,61,Berlín,1,Berlín,,Berlín,Embajada,1001,110046138042512901,Presidente y Vicepresidente,237,129,0,3,0,37,0,1,1,2,173,1,0,None,Berlín,173,0,0,63,63,174,NO,CC
19320,19321,2019-10-20 19:40:57,Bolivia,6,Arce,2,Bermejo,,Bermejo,U.E. Octavio Campero Echazu (Bermejo),60952,100320619160001408,Presidente y Vicepresidente,231,76,0,0,1,75,4,17,5,1,179,3,12,None,Tarija,179,0,0,37,37,194,NO,CC
19319,19320,2019-10-20 19:40:57,Bolivia,6,Arce,2,Bermejo,,Bermejo,U.E. Octavio Campero Echazu (Bermejo),60951,100320619160001407,Presidente y Vicepresidente,228,99,0,0,0,64,5,19,1,0,188,4,5,None,Tarija,188,0,0,31,31,197,NO,CC
19315,19316,2019-10-20 19:40:57,Bolivia,6,Arce,2,Bermejo,,Bermejo,U.E. Mscal. Andres De Santa Cruz,60944,100320619160001312,Presidente y Vicepresidente,215,79,1,0,0,53,4,22,3,0,162,1,5,None,Tarija,162,0,0,47,47,168,NO,CC
19314,19315,2019-10-20 19:40:57,Bolivia,6,Arce,2,Bermejo,,Bermejo,U.E. Mscal. Andres De Santa Cruz,60943,100320619160001311,Presidente y Vicepresidente,227,77,0,0,3,69,1,23,1,1,175,4,8,None,Tarija,175,0,0,40,40,187,NO,CC


In [39]:
df_llegada_actas_resumen = df_llegada_actas.groupby(by=['FechaHoraArchivo', 'Departamento', '_PartidoGanador']).count()[['CodigoMesa']].reset_index()
df_llegada_actas_resumen.head()

,FechaHoraArchivo,Departamento,_PartidoGanador,CodigoMesa
0,2019-10-20 19:40:57,Acre,CC,1
1,2019-10-20 19:40:57,Andalucia,CC,18
2,2019-10-20 19:40:57,Andalucia,MAS,4
3,2019-10-20 19:40:57,Antofagasta,CC,29
4,2019-10-20 19:40:57,Antofagasta,MAS,1


In [40]:
configure_plotly_browser_state()
df_llegada_actas_mas_cc = df_llegada_actas_resumen[df_llegada_actas_resumen['_PartidoGanador'].isin(['MAS','CC', 'PDC'])]
fig = px.bar(df_llegada_actas_mas_cc, x="FechaHoraArchivo",  title="Orden de llegada de actas", color= "_PartidoGanador" , text="Departamento")
fig.show()

In [41]:
df_llegada_actas_resumen_votos = df_llegada_actas.groupby(by=['FechaHoraArchivo', '_PartidoGanador', 'Provincia']).sum()[['MAS', 'CC']].reset_index()
df_llegada_actas_resumen_votos.head()

,FechaHoraArchivo,_PartidoGanador,Provincia,MAS,CC
0,2019-10-20 19:40:57,CC,Abel Iturralde,239,311
1,2019-10-20 19:40:57,CC,Abuna,163,257
2,2019-10-20 19:40:57,CC,Andres Ibañez,161205,429887
3,2019-10-20 19:40:57,CC,Angel Sandoval,64,69
4,2019-10-20 19:40:57,CC,Antofagasta,801,1700


In [42]:
configure_plotly_browser_state()
fig = px.bar(df_llegada_actas_resumen_votos, x="FechaHoraArchivo", y="MAS",  title="Orden de llegada de actas - Cantidad de votos MAS", color= "Provincia" )
fig.show()

In [43]:
configure_plotly_browser_state()
fig = px.bar(df_llegada_actas_resumen_votos, x="FechaHoraArchivo", y="CC",  title="Orden de llegada de actas - Cantidad de votos CC", color= "Provincia" )
fig.show()